In [19]:
import pandas as pd
import numpy as np

csv_schools=r'C:\Users\K Y\pandas_data_challenge\PyCitySchools\raw_data\schools_complete.csv'
csv_students=r'C:\Users\K Y\pandas_data_challenge\PyCitySchools\raw_data\students_complete.csv'


In [20]:
schoolsData = pd.read_csv(csv_schools)
students_df = pd.read_csv(csv_students)

#change name in schoolsData (output = schools_df) so it matches school in students_df
schoolsData = schoolsData.rename(columns = {'name':'school'})
schools_df = schoolsData.set_index('school', drop=True)
students_df.head(15)

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
5,5,Bryan Miranda,M,9th,Huang High School,94,94
6,6,Sheena Carter,F,11th,Huang High School,82,80
7,7,Nicole Baker,F,12th,Huang High School,96,69
8,8,Michael Roth,M,10th,Huang High School,95,87
9,9,Matthew Greene,M,10th,Huang High School,96,84


In [21]:
# ---District Summary ---
'''table of key district metrics including: Total Schools, Total Students, Total Budget, Average Math Score,
Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)'''

schoolCt = schools_df['School ID'].count()
studentCt = students_df['name'].count()
SumTotalBudg = schools_df['budget'].sum()
AvgMath = students_df['math_score'].mean()
AvgRead = students_df['reading_score'].mean()
#variables only used to get percentages
MathPassNum = students_df[students_df.math_score >= 60].count()
ReadPassNum = students_df[students_df.reading_score >= 60].count()
#getting percentages
MathPassPerc = (MathPassNum / studentCt) * 100
ReadPassPerc = (ReadPassNum / studentCt) * 100
OverallPass = (MathPassPerc + ReadPassPerc) / 2

#Setting up the table, 
DistMetric_df = pd.DataFrame({'Total Schools':schoolCt, 'Total Students':studentCt, 'Total Budget':SumTotalBudg,\
    'Average Math Score':AvgMath, 'Average Reading Score':AvgRead, '% Passing Math':MathPassPerc,\
    '% Passing Reading':ReadPassPerc, 'Overall Passing Rate':OverallPass})
#resetting index to numbers,
DistMetric_df.reset_index(drop=True, inplace = True)
#rearranging the order,
DistMetric_df = DistMetric_df[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score',\
    'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
#and getting first row of table
DistMetric_df.head(1)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,92.445749,100.0,96.222875


In [22]:
# ---School Summary---
'''merge schools_df with new variables: Average Math Score, Average Reading Score,
% Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)'''

#initializing groupby of students_df, getting student count and average scores for M and R
GroupStuSch = students_df.groupby('school')
StudentCt = GroupStuSch['Student ID'].count()
PerStuBudg = schools_df['budget'] / StudentCt
AvgScoresM = GroupStuSch['math_score'].mean()
AvgScoresR = GroupStuSch['reading_score'].mean()

#Getting the count of all passing students in both math and reading
SchoolPassM = students_df[students_df.math_score >= 60].groupby('school').count() 
SchoolPassR = students_df[students_df.reading_score >= 60].groupby('school').count()

#Getting pass % 
SchPerPassM = (SchoolPassM['math_score'] / StudentCt) * 100
SchPerPassR = (SchoolPassR['reading_score'] / StudentCt) * 100
SchOverall = (SchPerPassM + SchPerPassR) / 2

#concatenate schools_df
frames = [PerStuBudg, AvgScoresM, AvgScoresR, SchPerPassM, SchPerPassR, SchOverall]
SchScores= pd.concat(frames, axis=1)
SchScores.columns = ['Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']
#merge with schools_df, clean up, and format
SchSummary = schools_df.join(SchScores)
del SchSummary['School ID']
SchSummary = SchSummary.rename(columns = {'type':'School Type', 'size':'Total Students', 'budget':'Total School Budget'})
SchSummaryFrmt = SchSummary.sort_index(axis=0) #sort school name (aka. the index) in alphabetical order  
SchSummaryFrmt = SchSummaryFrmt.style.set_properties(**{'text-align': 'right'})
SchSummaryFrmt


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school,,,,,,,,,
Bailey High School,District,4976,3124928,628,77.0484,81.034,89.5297,100,94.7649
Cabrera High School,Charter,1858,1081356,582,83.0619,83.9758,100,100,100
Figueroa High School,District,2949,1884411,639,76.7118,81.158,88.4368,100,94.2184
Ford High School,District,2739,1763916,644,77.1026,80.7463,89.3027,100,94.6513
Griffin High School,Charter,1468,917500,625,83.3515,83.8168,100,100,100
Hernandez High School,District,4635,3022020,652,77.2898,80.9344,89.0831,100,94.5415
Holden High School,Charter,427,248087,581,83.8033,83.815,100,100,100
Huang High School,District,2917,1910635,655,76.6294,81.1827,88.8584,100,94.4292
Johnson High School,District,4761,3094650,650,77.0725,80.9664,89.1829,100,94.5915


In [23]:
# **Top Performing Schools (By Passing Rate)**
'''Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name * School Type * Total Students * Total School Budget * Per School Budget * Average Math Score
  * Average Reading Score * % Passing Math * % Passing Reading * Overall Passing Rate (Average of the above two)
'''
TopSchs = SchSummary.sort_values('Overall Passing Rate', ascending=False)
TopSchs.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school,,,,,,,,,
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,100.0,100.0,100.0
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.0,100.0,100.0
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,100.0,100.0,100.0
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,100.0,100.0,100.0
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,100.0,100.0,100.0


In [24]:
# **Bottom Performing Schools (By Passing Rate)**
'''Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name * School Type * Total Students * Total School Budget * Per School Budget * Average Math Score
  * Average Reading Score * % Passing Math * % Passing Reading * Overall Passing Rate (Average of the above two)
'''
BotSchs = SchSummary.sort_values('Overall Passing Rate')
BotSchs.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school,,,,,,,,,
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,88.436758,100.0,94.218379
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,88.547137,100.0,94.273568
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,88.858416,100.0,94.429208
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,89.083064,100.0,94.541532
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,89.182945,100.0,94.591472


In [25]:
# **Math Scores by Grade**
'''Create a table that lists the average Math Score for students of each grade level 
(9th, 10th, 11th, 12th) at each school.'''

#create pivot table of average math scores for each school by grade
tableM = pd.pivot_table(students_df, values='math_score', index=['school'], columns=['grade'], aggfunc=np.mean)
#--FYI: pivot_table's aggfunc argument defaults to mean
#fixing the order of the grade so that '9th' isn't last
grad_order = ['9th', '10th', '11th', '12th']
tableM_frmt = tableM.reindex(columns = grad_order)

tableM_frmt



grade,9th,10th,11th,12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [26]:
# **Reading Scores by Grade**
'''Create a table that lists the average Reading Score for students of each grade level
(9th, 10th, 11th, 12th) at each school.'''

#create pivot table of average reading scores for each school by grade
tableR = pd.pivot_table(students_df, values= 'reading_score', index='school', columns='grade')
#format so '9th' column isn't at the end, using previous list [grad_order]
tableR_frmt = tableR.reindex(columns = grad_order)
tableR_frmt


grade,9th,10th,11th,12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [32]:
# **Scores by School Spending**
'''Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
Use 4 reasonable bins to group school spending. Include in the table each of the following:
* Average Math Score * Average Reading Score * % Passing Math * % Passing Reading * Overall Passing Rate (Average of the above two)'''


SchSpendSumma = SchSummary[['Average Math Score', 'Average Reading Score', '% Passing Math',\
                            '% Passing Reading', 'Overall Passing Rate', 'Per Student Budget']].copy()


#creating bins and applying to new dataframe
binsSpPerStu = [0, 600, 625, 650, 675]
groupNameBin = ['<600', '600-625', '625-650', '650-675']

SchSpendSumma['Spending Ranges (Per Student)'] = pd.cut(SchSummary['Per Student Budget'], binsSpPerStu, right=False, labels=groupNameBin)


tableASPS = pd.pivot_table(SchSpendSumma, values = ['Average Math Score', 'Average Reading Score',\
    '% Passing Math', '% Passing Reading', 'Overall Passing Rate'], index = 'Spending Ranges (Per Student)' )
column_order = ['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',\
            'Overall Passing Rate']
tableASPS = tableASPS.reindex(columns = column_order)
tableASPS

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<600,83.455399,83.933814,100.000000,100.0,100.000000
600-625,83.599686,83.885211,100.000000,100.0,100.000000
625-650,79.079225,81.891436,92.636050,100.0,96.318025
650-675,76.997210,81.027843,89.041475,100.0,94.520737


In [35]:
# **Scores by School Size**
'''Repeat the above breakdown, but this time group schools based 
on a reasonable approximation of school size (Small, Medium, Large).'''
#Min/Max Values for SchSummary['Total Students']: max == 4976, min == 427
binsSchSize = [0, 1500, 3000, 5000]
groupNames = ['Small (<1500)', 'Medium (1500-3000)', 'Large (3000-5000)']

SchSizeSumma = SchSummary[['Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math',\
                            '% Passing Reading', 'Overall Passing Rate']].copy()

SchSizeSumma['School Size'] = pd.cut(SchSpendSumma['Total Students'], binsSchSize, right=False, labels=groupNames)

table_SchSize = pd.pivot_table(SchSizeSumma, values=['Average Math Score', 'Average Reading Score', '% Passing Math',\
                            '% Passing Reading', 'Overall Passing Rate'], index='School Size')

table_SchSize = table_SchSize.reindex(columns=column_order)
table_SchSize

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1500),83.664898,83.892148,100.000000,100.0,100.000000
Medium (1500-3000),80.904987,82.822740,95.824730,100.0,97.912365
Large (3000-5000),77.063340,80.919864,89.085722,100.0,94.542861


In [38]:
# **Scores by School Type**
#* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

#binsType = ['Charter', 'District']

SchTypeSumma = SchSummary[['School Type', 'Average Math Score', 'Average Reading Score', '% Passing Math',\
                            '% Passing Reading', 'Overall Passing Rate']].copy()
table_SchType = pd.pivot_table(SchTypeSumma, values = ['Average Math Score', 'Average Reading Score', '% Passing Math',\
                            '% Passing Reading', 'Overall Passing Rate'],\
                              index='School Type')
table_SchType = table_SchType.reindex(columns=column_order)
table_SchType

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,100.000000,100.0,100.000000
District,76.956733,80.966636,88.991533,100.0,94.495766
